In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './folder_to_save_graph_3' # Here you have to put your log directory
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [1]:
import os 
from glob import glob
import shutil 
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

In [2]:
data_dir = '../input/'
train_dir = data_dir + 'train/train/'
test_dir = data_dir + 'test/test'

input_shape = (32, 32, 3)
batch_size = 32
num_classes = 2
num_epochs = 20
data_augmentation = True

learning_rate = 0.001

In [3]:
# from tensorflow.keras.applications.vgg16 import VGG16

# base_model = VGG16(weights='imagenet',include_top=False)  # 우측 setting에서 internet을 켜야함

# net = base_model.output
# net = layers.GlobalAveragePooling2D()(net)
# net = layers.Dense(1024,activation='relu')(net)
# net = layers.Dense(1024,activation='relu')(net)
# net = layers.Dense(512,activation='relu')(net)
# net = layers.Dense(2,activation='softmax')(net)

# model = tf.keras.Model(inputs=base_model.input,outputs=net)

# model.layers

# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [4]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(64, (3, 3), padding='same')(inputs)
net = layers.Conv2D(64, (3, 3), padding='same')(net)
net = layers.Conv2D(64, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)

net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

# net = layers.GlobalAveragePooling2D()(net)
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])


df = pd.read_csv(data_dir + 'train.csv')
df.head()

#  (fname, cls), (fname, cls), (fname, cls), (fname, cls) (fname, cls)
datasets = []
for fname, cls in zip(df['id'], df['has_cactus']):
    datasets.append((fname, cls))

np.random.shuffle(datasets)
train_paths = datasets[:int(len(datasets) * 0.8)]
test_paths = datasets[int(len(datasets) * 0.8):]

# train_rate = 0.8
# np.random.shuffle(datasets)
# num_trains = int(len(datasets) * train_rate)

# train_paths = datasets[:num_trains]
# test_paths = datasets[num_trains:]

In [6]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

def batch_dataset(batch_paths):
    batch_images = []
    batch_labels = []

    for fname, cls in batch_paths:
        img_path = os.path.join(train_dir, fname)
        image = np.array(Image.open(img_path)) / 255.
        label = np.array(np.array([0, 1]) == cls).astype(np.uint8)
        batch_images.append(image)
        batch_labels.append(label)

    batch_images = np.array(batch_images)
    batch_labels = np.array(batch_labels)
    
    return batch_images, batch_labels

In [7]:
def data_gen(data_paths, is_training=True):
    global_step = 0
    steps_per_epoch = len(data_paths) // batch_size
    while True:
        step = global_step % steps_per_epoch
        images, labels = batch_dataset(data_paths[step*batch_size: (step+1)*batch_size])
        datagen.fit(images)
        images, labels = next(datagen.flow(images, labels)) if is_training else (images, labels)
        global_step += 1
        
        yield images, labels

In [8]:
callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath='my_model.h5', # 저장
#         monitor='val_loss',
#         save_best_only=True, # 가장 좋은 모델
#     ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_acc',
        factor=0.1, # 콜백이 호출되면 학습률을 10배로 줄임
        patience=2, # 몇 번 확인 후 안 오르면 학습률을 줄임
    ),
    # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    # `./logs` 디렉토리에 텐서보드 로그를 기록니다.
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir logs

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


Launching TensorBoard...

In [ ]:
for epoch in range(num_epochs):
    np.random.shuffle(train_paths)
    steps_per_epoch = len(train_paths) // batch_size
    
    history = model.fit_generator(data_gen(train_paths), 
                                  steps_per_epoch=steps_per_epoch, 
                                  validation_data=batch_dataset(test_paths[:batch_size]),
                                  verbose=1,
                                  callbacks=callbacks
                                  )

Instructions for updating:
Use tf.cast instead.


In [ ]:
test_batches_per_epoch = len(test_paths) // batch_size
model.evaluate_generator(data_gen(test_paths, False), 
                         steps=5, 
                         verbose=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
image, label = next(data_gen(test_paths, False))

logits = model.predict(image)

In [ ]:
confusion_matrix(np.argmax(logits, -1), np.argmax(label, -1))  # (tn, fp), (fn, tp)

In [ ]:
(tn, fp), (fn, tp) = confusion_matrix(np.argmax(logits, -1), np.argmax(label, -1))

In [ ]:
test_dir = data_dir + 'test/test'
test_df = pd.read_csv(data_dir + 'sample_submission.csv')
test_df['id'][0]

answers = []
for fname in tqdm_notebook(test_df['id']):
    path = os.path.join(test_dir, fname)
    image = np.array(Image.open(path)) / 255.
    image = np.expand_dims(image, 0)
    logit = model.predict(image)
    answers.append(logit[0])

submit_data = {'id': test_df['id'],
               'has_cactus': np.argmax(answers, -1)}

submit_df = pd.DataFrame(submit_data)
submit_df.to_csv('samplesubmission.csv', index=False)